## Data Wrangling

- fix invalid player IDs who played in games (Mintues Played != DNP)

### 2022-23 Season

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
df = pd.read_csv('../data/processed/Season(2023-24).csv')
df.head()

,Date,Name,Team,MP,FG,FGA,FG%,3P,3PA,3P%,...,PTS,GmSc,+-,TeamAbbr,GameLink,Opponent,Home,GameType,id,fpts_fanduel
0,20231024,d'angelo russell,Los Angeles Lakers,36:11,4,12,.333,2,5,.400,...,11,6.7,+1,LAL,https://www.basketball-reference.com/boxscores...,Denver Nuggets,0,Season,20001458,26.3
1,20231024,anthony davis,Los Angeles Lakers,34:09,6,17,.353,1,2,.500,...,17,11.3,-17,LAL,https://www.basketball-reference.com/boxscores...,Denver Nuggets,0,Season,20000468,36.6
2,20231024,austin reaves,Los Angeles Lakers,31:20,4,11,.364,1,2,.500,...,14,13.1,-14,LAL,https://www.basketball-reference.com/boxscores...,Denver Nuggets,0,Season,20002720,33.6
3,20231024,taurean prince,Los Angeles Lakers,29:53,6,8,.750,4,6,.667,...,18,16.5,-14,LAL,https://www.basketball-reference.com/boxscores...,Denver Nuggets,0,Season,20001680,25.1
4,20231024,lebron james,Los Angeles Lakers,29:00,10,16,.625,1,4,.250,...,21,20.3,+7,LAL,https://www.basketball-reference.com/boxscores...,Denver Nuggets,0,Season,20000571,41.1


In [4]:
invalid_id_players = df[(df['id'] == -1) & (df['MP'] != 'DNP')]
player_count = len(invalid_id_players)
print("\nNumber of players with invalid id and MP != 'DNP':", player_count)


Number of players with invalid id and MP != 'DNP': 642


In [47]:
invalid_id_players = df[(df['id'] == -1) & (df['MP'] != 'DNP')]
player_counts = invalid_id_players.groupby('Name').size()
first_game_date = invalid_id_players.groupby('Name')['Date'].min()

player_counts_df = player_counts.reset_index(name='Count').sort_values(by='Count', ascending=False)
first_game_date_df = first_game_date.reset_index(name='FirstGameDate')
player_counts_with_date_df = pd.merge(player_counts_df, first_game_date_df, on='Name')

total_row = pd.DataFrame([['Total', player_counts_with_date_df['Count'].sum(), '']], columns=['Name', 'Count', 'FirstGameDate'])
player_counts_with_date_df = pd.concat([player_counts_with_date_df, total_row], ignore_index=True)

print(player_counts_with_date_df)

                    Name  Count FirstGameDate
0        dennis schroder     80      20231025
1            nic claxton     71      20231025
2         alperen sengun     63      20231025
3             a.j. green     62      20231029
4              kj martin     60      20231025
5          marcus morris     58      20231106
6         omer yurtseven     48      20231025
7             kevin knox     31      20231112
8            harry giles     23      20231110
9             vit krejci     22      20240214
10    jeenathan williams     22      20231025
11          theo maledon     17      20231025
12      jermaine samuels     14      20231025
13           jeff dowtin     12      20240305
14          e.j. liddell      9      20231028
15          matthew hurt      8      20240129
16       robert williams      6      20231025
17        isaiah jackson      2      20231209
18            seth curry      2      20231122
19           isaiah wong      1      20231209
20           cam reddish      1   

,Date,Name,Team,MP,FG,FGA,FG%,3P,3PA,3P%,...,PTS,GmSc,+-,TeamAbbr,GameLink,Opponent,Home,GameType,id,fpts_fanduel
252,20231025,harry giles,Brooklyn Nets,DNP,DNP,DNP,DNP,DNP,DNP,DNP,...,DNP,DNP,DNP,BRK,https://www.basketball-reference.com/boxscores...,Cleveland Cavaliers,1,Season,-1,0.0
661,20231027,harry giles,Brooklyn Nets,DNP,DNP,DNP,DNP,DNP,DNP,DNP,...,DNP,DNP,DNP,BRK,https://www.basketball-reference.com/boxscores...,Dallas Mavericks,0,Season,-1,0.0
1187,20231030,harry giles,Brooklyn Nets,DNP,DNP,DNP,DNP,DNP,DNP,DNP,...,DNP,DNP,DNP,BRK,https://www.basketball-reference.com/boxscores...,Charlotte Hornets,0,Season,-1,0.0
1623,20231101,harry giles,Brooklyn Nets,DNP,DNP,DNP,DNP,DNP,DNP,DNP,...,DNP,DNP,DNP,BRK,https://www.basketball-reference.com/boxscores...,Miami Heat,0,Season,-1,0.0
2027,20231103,harry giles,Brooklyn Nets,DNP,DNP,DNP,DNP,DNP,DNP,DNP,...,DNP,DNP,DNP,BRK,https://www.basketball-reference.com/boxscores...,Chicago Bulls,0,Season,-1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27452,20240324,harry giles,Los Angeles Lakers,DNP,DNP,DNP,DNP,DNP,DNP,DNP,...,DNP,DNP,DNP,LAL,https://www.basketball-reference.com/boxscores...,Indiana Pacers,1,Season,-1,0.0
27757,20240326,harry giles,Los Angeles Lakers,DNP,DNP,DNP,DNP,DNP,DNP,DNP,...,DNP,DNP,DNP,LAL,https://www.basketball-reference.com/boxscores...,Milwaukee Bucks,0,Season,-1,0.0
28010,20240327,harry giles,Los Angeles Lakers,5:08,0,3,.000,0,2,.000,...,0,-1.1,-12,LAL,https://www.basketball-reference.com/boxscores...,Memphis Grizzlies,0,Season,-1,0.0
31115,20240412,harry giles,Los Angeles Lakers,DNP,DNP,DNP,DNP,DNP,DNP,DNP,...,DNP,DNP,DNP,LAL,https://www.basketball-reference.com/boxscores...,Memphis Grizzlies,0,Season,-1,0.0


In [48]:
df_22_23 = pd.read_csv('../data/processed/Season(2022-23).csv')
df_23_24 = pd.read_csv('../data/processed/Season(2023-24).csv')

In [49]:
id_mapping_22_23 = {
    'zeke nnaji': 20002547,
    'willy hernangomez': 20001405,
    'taj gibson': 20000517,
    'kessler edwards': 20002711,
    'frank kaminsky': 20001396,
    'jeenathan williams': 20002948,
    'omer yurtseven': 20002599,
    'john butler': 20002898,
    'jeff dowtin': 20002786,
    'vit krejci': 20002562,
    'a.j. green': 20002945,
    'theo maledon': 20002559,
    'robert williams': 20002024,
    'kevin knox': 20001994,
    'marcus morris': 20000847,
    'alperen sengun': 20002666,
    'nic claxton': 20002292,
    'kj martin': 20002577,
    'dennis schroder': 20000606
}

id_mapping_23_24 = {
    'tyrese haliburton': 20002537,
    'trayce jackson-davis': 20003065,
    't.j. mcconnell': 20001525,
    'taurean prince': 20001680,
    'rui hachimura': 20002275,
    'maxwell lewis': 20003057,
    'obi toppin': 20002533,
    'oscar tshiebwe': 20003130,
    'max christie': 20002900,
    'myles turner': 20001424,
    'jarred vanderbilt': 20002015,
    'jaxson hayes': 20002274,
    'kendall brown': 20002869,
    'justin holiday': 20000492,
    'jontay porter': 20002444,
    'lebron james': 20000571,
    'jalen hood-schifino': 20003045,
    'jay huff': 20002735,
    'jarace walker': 20003035,
    'aaron nesmith': 20002539,
    'anthony davis': 20000468,
    'austin reaves': 20002720,
    'ben sheppard': 20003070,
    'bennedict mathurin': 20002879,
    'buddy hield': 20001673,
    'bruce brown': 20002040,
    'bol bol': 20002291,
    'bismack biyombo': 20000758,
    'd\'angelo russell': 20001458,
    'cody zeller': 20000754,
    'cam reddish': 20002276,
    'isaiah wong': 20003088,
    'seth curry': 20000933,
    'isaiah jackson': 20002673,
    'robert williams': 20002024,
    'matthew hurt': 20002679,
    'e.j. liddell': 20002887,
    'jeff dowtin': 20002786,
    'jermaine samuels': 20002947,
    'theo maledon': 20002559,
    'jeenathan williams': 20002948,
    'vit krejci': 20002562,
    'harry giles': 20001827,
    'kevin knox': 20001994,
    'omer yurtseven': 20002599,
    'marcus morris': 20000847,
    'kj martin': 20002577,
    'a.j. green': 20002945,
    'alperen sengun': 20002666,
    'nic claxton': 20002292,
    'dennis schroder': 20000606
}

### Fix IDs for each player

In [51]:
def update_ids(df, id_mapping):
    df['Name_lower'] = df['Name'].str.lower()
    df['id'] = df.apply(lambda x: id_mapping[x['Name_lower']] if x['Name_lower'] in id_mapping else x['id'], axis=1)
    df.drop(columns=['Name_lower'], inplace=True)
    return df

season_22_23 = update_ids(df_22_23, id_mapping_22_23)
season_23_24 = update_ids(df_23_24, id_mapping_23_24)

season_22_23.to_csv('../data/processed/Season(2022-23)_updated.csv', index=False)
season_23_24.to_csv('../data/processed/Season(2023-24)_updated.csv', index=False)

In [56]:
print(season_22_23.shape, season_23_24.shape)

(33606, 31) (34126, 31)


In [52]:
invalid_id_players = season_22_23[(season_22_23['id'] == -1) & (season_22_23['MP'] != 'DNP')]
player_count = len(invalid_id_players)
print("\nNumber of players with invalid id and MP != 'DNP':", player_count)


Number of players with invalid id and MP != 'DNP': 0


In [53]:
invalid_id_players = season_23_24[(season_23_24['id'] == -1) & (season_23_24['MP'] != 'DNP')]
player_count = len(invalid_id_players)
print("\nNumber of players with invalid id and MP != 'DNP':", player_count)


Number of players with invalid id and MP != 'DNP': 0


In [87]:
invalid_id_players = season_22_23[(season_22_23['id'] == -1) & (season_22_23['MP'] != 'DNP')]
player_counts = invalid_id_players.groupby('Name').size()
first_game_date = invalid_id_players.groupby('Name')['Date'].min()

player_counts_df = player_counts.reset_index(name='Count').sort_values(by='Count', ascending=False)
first_game_date_df = first_game_date.reset_index(name='FirstGameDate')
player_counts_with_date_df = pd.merge(player_counts_df, first_game_date_df, on='Name')

total_row = pd.DataFrame([['Total', player_counts_with_date_df['Count'].sum(), '']], columns=['Name', 'Count', 'FirstGameDate'])
player_counts_with_date_df = pd.concat([player_counts_with_date_df, total_row], ignore_index=True)

print(player_counts_with_date_df)

    Name  Count FirstGameDate
0  Total      0              


In [86]:
invalid_id_players = season_23_24[(season_23_24['id'] == -1) & (season_23_24['MP'] != 'DNP')]
player_counts = invalid_id_players.groupby('Name').size()
first_game_date = invalid_id_players.groupby('Name')['Date'].min()

player_counts_df = player_counts.reset_index(name='Count').sort_values(by='Count', ascending=False)
first_game_date_df = first_game_date.reset_index(name='FirstGameDate')
player_counts_with_date_df = pd.merge(player_counts_df, first_game_date_df, on='Name')

total_row = pd.DataFrame([['Total', player_counts_with_date_df['Count'].sum(), '']], columns=['Name', 'Count', 'FirstGameDate'])
player_counts_with_date_df = pd.concat([player_counts_with_date_df, total_row], ignore_index=True)

print(player_counts_with_date_df)

    Name  Count FirstGameDate
0  Total      0              


In [95]:
season_22_23[season_22_23['MP'] == 'DNP']['MP'].value_counts()

MP
DNP    5867
Name: count, dtype: int64

In [97]:
season_23_24[season_23_24['MP'] == 'DNP']['MP'].value_counts()

MP
DNP    5895
Name: count, dtype: int64

In [96]:
print(season_22_23[season_22_23['MP'] != 'DNP'].shape)
print(27739+5867)

(27739, 31)
33606


In [99]:
print(season_23_24[season_23_24['MP'] != 'DNP'].shape)
print(28231+5895)

(28231, 31)
34126


In [101]:
season_22_23_cleaned = season_22_23[season_22_23['MP'] != 'DNP']
season_23_24_cleaned = season_23_24[season_23_24['MP'] != 'DNP']

season_22_23_cleaned.to_csv('../data/processed/Season(2022-23)_cleaned.csv', index=False)
season_23_24_cleaned.to_csv('../data/processed/Season(2023-24)_cleaned.csv', index=False)

In [108]:
print(season_22_23_cleaned[season_22_23_cleaned['id'] == -1].shape[0])
print(season_23_24_cleaned[season_23_24_cleaned['id'] == -1].shape[0])
print(season_22_23_cleaned.shape)
print(season_23_24_cleaned.shape)

0
0
(27739, 31)
(28231, 31)


## Format Data to correct data types

In [159]:
df = pd.read_csv('../data/processed/Season(2022-23)_cleaned.csv')

In [160]:
df.dtypes

Unnamed: 0        int64
Date             object
Name             object
Team             object
MP               object
FG                int64
FGA               int64
FG%             float64
3P                int64
3PA               int64
3P%             float64
FT                int64
FTA               int64
FT%             float64
ORB               int64
DRB               int64
TRB               int64
AST               int64
STL               int64
BLK               int64
TOV               int64
PF                int64
PTS               int64
GmSc            float64
+-                int64
TeamAbbr         object
GameLink         object
Opponent         object
Home              int64
GameType         object
id                int64
fpts_fanduel    float64
dtype: object

In [161]:
df['Date'] = pd.to_datetime(df['Date'])

In [162]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [163]:
df.dtypes

Date            datetime64[ns]
Name                    object
Team                    object
MP                      object
FG                       int64
FGA                      int64
FG%                    float64
3P                       int64
3PA                      int64
3P%                    float64
FT                       int64
FTA                      int64
FT%                    float64
ORB                      int64
DRB                      int64
TRB                      int64
AST                      int64
STL                      int64
BLK                      int64
TOV                      int64
PF                       int64
PTS                      int64
GmSc                   float64
+-                       int64
TeamAbbr                object
GameLink                object
Opponent                object
Home                     int64
GameType                object
id                       int64
fpts_fanduel           float64
dtype: object

In [164]:
df.head()

,Date,Name,Team,MP,FG,FGA,FG%,3P,3PA,3P%,...,PTS,GmSc,+-,TeamAbbr,GameLink,Opponent,Home,GameType,id,fpts_fanduel
0,2022-10-18,tyrese maxey,Philadelphia 76ers,38:11,8,16,0.500,2,5,0.400,...,21,13.7,-6,PHI,https://www.basketball-reference.com/boxscores...,Boston Celtics,0,Season,20002546,30.2
1,2022-10-18,james harden,Philadelphia 76ers,37:17,9,14,0.643,5,9,0.556,...,35,31.9,1,PHI,https://www.basketball-reference.com/boxscores...,Boston Celtics,0,Season,20000544,52.1
2,2022-10-18,joel embiid,Philadelphia 76ers,37:16,9,18,0.500,1,6,0.167,...,26,17.7,-13,PHI,https://www.basketball-reference.com/boxscores...,Boston Celtics,0,Season,20000726,48.5
3,2022-10-18,tobias harris,Philadelphia 76ers,34:14,7,14,0.500,3,6,0.500,...,18,13.4,-1,PHI,https://www.basketball-reference.com/boxscores...,Boston Celtics,0,Season,20000676,29.4
4,2022-10-18,p.j. tucker,Philadelphia 76ers,33:01,3,5,0.600,0,2,0.000,...,6,3.6,-6,PHI,https://www.basketball-reference.com/boxscores...,Boston Celtics,0,Season,20000856,11.8


In [165]:
df = df.to_csv('../data/processed/Season(2022-23)_cleaned.csv', index=False)